## Here I will be using two test documents containing similar paragraphs and test the similarity scores using different techniques

In [20]:
d1 = open('doc1.txt').read()
d2 = open('doc2.txt').read()

In [21]:
# document list can extended as per your suite
documents = [d1, d2]

In [22]:
import nltk, string, numpy
#nltk.download('wordnet') # first-time use only

In [23]:
lemmer = nltk.stem.WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

# Document Similarity using tf-idf

Here, I define term frequency-inverse document frequency (tf-idf) vectorizer parameters and then convert the synopses list into a tf-idf matrix.

To get a Tf-idf matrix, first count word occurrences by document. This is transformed into a document-term matrix (dtm). This is also just called a term frequency matrix. An example of a dtm is here at right.

Then apply the term frequency-inverse document frequency weighting: words that occur frequently within a document but not frequently within the corpus receive a higher weighting as these words are assumed to contain more meaning in relation to the document.

A couple things to note about the parameters I define below:

max_df: this is the maximum frequency within the documents a given feature can have to be used in the tfi-idf matrix. If the term is in greater than 80% of the documents it probably cares little meanining (in the context of film synopses)
min_idf: this could be an integer (e.g. 5) and the term would have to be in at least 5 of the documents to be considered. Here I pass 0.2; the term must be in at least 20% of the document. I found that if I allowed a lower min_df I ended up basing clustering on names--for example "Michael" or "Tom" are names found in several of the movies and the synopses use these names frequently, but the names carry no real meaning.
ngram_range: this just means I'll look at unigrams, bigrams and trigrams. See n-grams

![title](http://www.jiem.org/index.php/jiem/article/viewFile/293/252/2402)

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer


TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')

def cos_similarity(textlist):
    tfidf = TfidfVec.fit_transform(textlist)
    #return (tfidf * tfidf.T).toarray()
    return tfidf

![title](https://lh4.googleusercontent.com/SodVc3Xo77b8LhEjqXymSaA-bI-kQdPeY8uG-J0wSSp5q-pxVAf_rPMUX9Y)

from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

In [25]:
tfidf = cos_similarity(documents)

#print(cos_similarity(documents))

from sklearn.metrics.pairwise import cosine_similarity
dist = cosine_similarity(tfidf)

print("Cosine Similarity: ",dist)

Cosine Similarity:  [[1.         0.23652373]
 [0.23652373 1.        ]]


/media/aayush/New-Volume/A_property/stuff_test/stuff_test/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


# ---------------------------------------------------------------------------------------------------------

# Using Doc2Vec